In [ ]:
from unsloth import FastLanguageModel
import torch
import pandas as pd 
import re
import json
max_seq_length = 2048 
dtype = torch.float16
load_in_4bit = False 


In [ ]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/cluster/work/users/anonymous/NProStrt_2_Llama-3.2-1B_128", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = False,
)
FastLanguageModel.for_inference(model)

In [ ]:
alpaca_prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""



inputss = tokenizer(
[
    alpaca_prompt.format(
        "Extract the information in JSONL format for the following source_text:", # instruction
        "Al-Hawl camp | Two women killed in south-eastern Al-Hasakah countryside SOHR sources have reported a new murder took place in Al-Hawl “mini-state” in the far south-eastern countryside of Al-Hasakah, when a displaced Syrian woman and unidentified woman were killed. According to SOHR sources, Internal Security Forces “Asayish” found the body of two women in the third section of the camp. This comes in light of ongoing attacks carried out by ISIS cells in Al-Hawl camp, which houses ISIS families and members. Accordingly, the number of murders documented in the camp since early 2022 has risen to 19, which left 20 people dead; and they are as follows: • Six Iraqi refugees, including two women. • Seven Syrian people: Two men and five women. • Six unidentified women. • A medic was killed in the post where he worked in the camp.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

In [ ]:
dataFrame = pd.read_json("/cluster/work/users/anonymous/new_jsons_prompt/test_data_json_prompt2.json")
list_out = []
for index, row in dataFrame.iterrows():
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            str(row['instruction']), # instruction
            str(row['input'][0:2000]), # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 200, use_cache = True)
    response_text = tokenizer.batch_decode(outputs)
    list_out.append("id: "+str(row['id'])+" "+ "".join( response_text))
    print("id: "+str(row['id'])+" "+ "".join( response_text))
print(list_out[0])
with open('output_NProStrt_2_Llama-3.2-1B_128.txt', 'w') as f:
    for line in list_out:
        f.write(f"{line}\n\t")